# Housing Affordability in Vancouver
### Rent vs Income Analysis (2019–2024)

This notebook analyzes housing affordability in Vancouver by comparing
rental price trends with income growth using a rent-to-income ratio.


## Analysis Scope

- Geography: Vancouver Census Metropolitan Area (CMA)
- Time Period: 2019–2024
- Metric: Rent-to-Income Ratio
- Affordability Threshold: 30%


In [9]:
import pandas as pd
import matplotlib.pyplot as plt


In [10]:
rent_df = pd.read_csv("../data/processed/vancouver_quarterly_rent_clean.csv")
income_df = pd.read_csv("../data/processed/vancouver_median_income_clean.csv")


In [11]:
rent_df.head()
income_df.head()


,year,median_income
0,2019,43300
1,2020,45800
2,2021,44800
3,2022,45900
4,2023,46300


In [12]:
rent_df.columns

Index(['Geography', 'Rental unit type', 'quarter', 'avg_rent'], dtype='object')

In [13]:
income_df.columns

Index(['year', 'median_income'], dtype='object')

In [14]:
rent_df.shape, income_df.shape

((27, 4), (5, 2))

In [15]:
rent_df["year"] = pd.to_datetime(rent_df["quarter"]).dt.year

C:\Users\Shibi\AppData\Local\Temp\ipykernel_8120\551750247.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  rent_df["year"] = pd.to_datetime(rent_df["quarter"]).dt.year


DateParseError: Unknown datetime string format, unable to parse: Q1 2019, at position 0

In [ ]:
# 1) Inspect the weird values (sanity check)
rent_df["quarter"].head(10)


In [ ]:
# 2) Extract quarter number and year from strings like "Q1 2019"
q = rent_df["quarter"].astype(str).str.extract(r"Q([1-4])\s*(\d{4})")

rent_df["q"] = q[0].astype("Int64")     # 1-4
rent_df["year"] = q[1].astype("Int64")  # 2019, 2020, ...


In [ ]:
rent_df[["quarter", "q", "year"]].head(10)
rent_df[["q", "year"]].isna().sum()


In [ ]:
rent_df["period"] = pd.PeriodIndex(year=rent_df["year"], quarter=rent_df["q"], freq="Q").to_timestamp()
rent_df[["quarter", "period"]].head(10)


In [ ]:
rent_df[["quarter","q","year"]].head(10)

rent_df[["q","year"]].isna().sum()

In [ ]:
rent_df["annual_rent"] = rent_df["avg_rent"] * 12


In [ ]:
rent_df[["avg_rent", "annual_rent"]].head()
rent_df["annual_rent"].describe()


In [ ]:
annual_rent_df = (
    rent_df
    .groupby("year", as_index=False)["annual_rent"]
    .mean()
)


In [ ]:
annual_rent_df


In [ ]:
income_df.head()
income_df.columns


In [ ]:
annual_income_df = (
    income_df
    .groupby("year", as_index=False)["avg_income"]
    .mean()
)


In [ ]:
income_df.head()


In [ ]:
income_df.columns


In [ ]:
annual_income_df = (
    income_df
    .groupby("year", as_index=False)["median_income"]
    .mean()
)


In [ ]:
annual_income_df


In [ ]:
annual_rent_df = annual_rent_df[annual_rent_df["year"].between(2019, 2023)]
annual_income_df = annual_income_df[annual_income_df["year"].between(2019, 2023)]


In [ ]:
annual_rent_df
annual_income_df


In [ ]:
affordability_df = pd.merge(
    annual_rent_df,
    annual_income_df,
    on="year",
    how="inner"
)


In [ ]:
affordability_df


In [ ]:
affordability_df["rent_to_income_ratio"] = (
    affordability_df["annual_rent"] / affordability_df["median_income"]
)


In [ ]:
affordability_df
affordability_df["rent_to_income_ratio"].describe()


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(
    affordability_df["year"],
    affordability_df["rent_to_income_ratio"],
    marker="o"
)

plt.axhline(0.30, linestyle="--")

plt.title("Vancouver Housing Affordability: Rent-to-Income Ratio")
plt.xlabel("Year")
plt.ylabel("Rent-to-Income Ratio")

plt.show()


In [ ]:
# Rent-to-Income Ratio Analysis for Vancouver CMA (2019–2024)

import pandas as pd
import matplotlib.pyplot as plt

# Load processed datasets
rent_path = "../data/processed/rent_vancouver_cma.csv"
income_path = "../data/processed/income_vancouver_cma.csv"

rent_df = pd.read_csv(rent_path, parse_dates=["quarter"])
income_df = pd.read_csv(income_path, parse_dates=["quarter"])

# Aggregate rent across unit types (median proxy via mean of averages)
rent_q = (
    rent_df
    .groupby("quarter", as_index=False)["avg_rent"]
    .mean()
    .rename(columns={"avg_rent": "avg_rent_all_units"})
)

# Merge with income
merged = pd.merge(rent_q, income_df, on="quarter", how="inner")

# Compute monthly income if income is annual
if merged["avg_income"].median() > 20000:
    merged["monthly_income"] = merged["avg_income"] / 12
else:
    merged["monthly_income"] = merged["avg_income"]

# Rent-to-income ratio
merged["rent_to_income_ratio"] = merged["avg_rent_all_units"] / merged["monthly_income"]

# Save output
out_path = "../data/processed/rent_to_income_vancouver_cma.csv"
merged.to_csv(out_path, index=False)

# Plot
plt.figure()
plt.plot(merged["quarter"], merged["rent_to_income_ratio"])
plt.axhline(0.30)
plt.title("Vancouver CMA Rent-to-Income Ratio (2019–2024)")
plt.xlabel("Quarter")
plt.ylabel("Rent-to-Income Ratio")
plt.tight_layout()
plt.show()

merged.head()
